# SQL

이번 튜토리얼은 `create_sql_query_chain` 을 활용하여 생성한 체인으로 SQL 쿼리를 생성 후 실행, 그리고 답변 도출하는 방법에 대해서 다룹니다. 그리고, SQL Agent 와의 동작 방식의 차이도 알아보겠습니다.

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("SQL")

LangSmith 추적을 시작합니다.
[프로젝트명]
SQL


SQL Database 정보를 불러옵니다.

In [3]:
from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase

# SQLite 데이터베이스에 연결합니다.
db = SQLDatabase.from_uri("sqlite:///data/finance.db")

# 데이터베이스의 dialect를 출력합니다.
print(db.dialect)

# 사용 가능한 테이블 이름들을 출력합니다.
print(db.get_usable_table_names())

sqlite
['accounts', 'customers', 'transactions']


LLM 객체를 생성하고 LLM 과 DB 를 매개변수로 입력하여 chain 을 생성합니다.

여기서 model 변경시 원활하게 동작하지 않을 수 있어, 이번 튜토리얼은 `gpt-3.5-turbo` 로 진행합니다.

In [4]:
# model 은 gpt-3.5-turbo 를 지정
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# LLM 과 DB 를 매개변수로 입력하여 chain 을 생성합니다.
chain = create_sql_query_chain(llm, db)

(옵션) 아래의 방식으로 Prompt 를 직접 지정할 수 있습니다.

직접 작성시 table_info 와 더불어 설명가능한 column description 을 추가할 수 있습니다.

In [19]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. Unless the user specifies in his question a specific number of examples he wishes to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:
{table_info}

Here is the description of the columns in the tables:
`cust`: customer name
`prod`: product name
`trans`: transaction date

Question: {input}"""
).partial(dialect=db.dialect)

# model 은 gpt-3.5-turbo 를 지정
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# LLM 과 DB 를 매개변수로 입력하여 chain 을 생성합니다.
chain = create_sql_query_chain(llm, db, prompt)

In [ ]:
import inspect

print(inspect.getsource(create_sql_query_chain))  # top_k 기본값은 5

def create_sql_query_chain(
    llm: BaseLanguageModel,
    db: SQLDatabase,
    prompt: Optional[BasePromptTemplate] = None,
    k: int = 5,
) -> Runnable[Union[SQLInput, SQLInputWithTables, Dict[str, Any]], str]:
    """Create a chain that generates SQL queries.

    *Security Note*: This chain generates SQL queries for the given database.

        The SQLDatabase class provides a get_table_info method that can be used
        to get column information as well as sample data from the table.

        To mitigate risk of leaking sensitive data, limit permissions
        to read and scope to the tables that are needed.

        Optionally, use the SQLInputWithTables input type to specify which tables
        are allowed to be accessed.

        Control access to who can submit requests to this chain.

        See https://python.langchain.com/docs/security for more information.

    Args:
        llm: The language model to use.
        db: The SQLDatabase to generate the query for.
     

chain 을 실행하면 DB 기반으로 쿼리를 생성합니다.

In [6]:
# chain 을 실행하고 결과를 출력합니다.
generated_sql_query = chain.invoke({"question": "고객의 이름을 나열하세요"})

# 생성된 쿼리를 출력합니다.
print(generated_sql_query.__repr__())

'SELECT name\nFROM customers'


다음은 생성한 쿼리가 맞게 동작하는지 확인해 볼 차례입니다.

In [7]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

# 생성한 쿼리를 실행하기 위한 도구를 생성합니다.
execute_query = QuerySQLDataBaseTool(db=db)

C:\Users\susis\AppData\Local\Temp\ipykernel_9544\3803918128.py:4: LangChainDeprecationWarning: The class `QuerySQLDataBaseTool` was deprecated in LangChain 0.3.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-community package and should be used instead. To use it run `pip install -U :class:`~langchain-community` and import as `from :class:`~langchain_community.tools import QuerySQLDatabaseTool``.
  execute_query = QuerySQLDataBaseTool(db=db)


In [8]:
execute_query.invoke({"query": generated_sql_query})

"[('테디',), ('폴',), ('셜리',), ('민수',), ('지영',), ('은정',)]"

In [20]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

# 도구
execute_query = QuerySQLDataBaseTool(db=db)

# SQL 쿼리 생성 체인
write_query = create_sql_query_chain(llm, db, prompt)

# 생성한 쿼리를 실행하기 위한 체인을 생성합니다.
chain = write_query | execute_query

In [21]:
# 실행 결과 확인
chain.invoke({"question": "테디의 이메일을 조회하세요"})

"[('teddy@example.com',)]"

## 답변을 LLM 으로 증강-생성

이전 단계에서 생성한 chain 을 사용하면 답변이 단답형 형식으로 출력됩니다. 이를 LCEL 문법의 체인으로 좀 더 자연스러운 답변을 받을 수 있도록 조정할 수 있습니다.

In [11]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()

# 생성한 쿼리를 실행하고 결과를 출력하기 위한 체인을 생성합니다.
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

In [16]:
# 실행 결과 확인
chain.invoke({"question": "테디의 transaction 의 합계를 구하세요."})

'테디의 transaction의 합계는 -965.7 입니다.'

In [ ]:
runnable = RunnablePassthrough.assign(query=write_query).assign(
    result=itemgetter("query") | execute_query
)

In [14]:
runnable.invoke({"question": "테디의 transaction 의 합계를 구하세요"})

{'question': '테디의 transaction 의 합계를 구하세요',
 'query': 'SELECT SUM(amount) AS total_transactions\nFROM transactions\nJOIN accounts ON transactions.account_id = accounts.account_id\nJOIN customers ON accounts.customer_id = customers.customer_id\nWHERE customers.name = "테디"',
 'result': '[(-965.7,)]'}

## Agent

Agent를 활용하여 Sql 쿼리를 생성하고 실행 결과를 답변으로 출력이 가능합니다.

In [17]:
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent

# model 은 gpt-3.5-turbo 를 지정
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# SQLite 데이터베이스에 연결합니다.
db = SQLDatabase.from_uri("sqlite:///data/finance.db")

# Agent 생성
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [18]:
# 실행 결과 확인
agent_executor.invoke(
    {"input": "테디와 셜리의 transaction 의 합계를 구하고 비교하세요"}
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


accounts, customers, transactions
Invoking: `sql_db_schema` with `{'table_names': 'transactions'}`



CREATE TABLE transactions (
	transaction_id INTEGER, 
	account_id INTEGER, 
	amount REAL, 
	transaction_date TEXT, 
	PRIMARY KEY (transaction_id), 
	FOREIGN KEY(account_id) REFERENCES accounts (account_id)
)

/*
3 rows from transactions table:
transaction_id	account_id	amount	transaction_date
1	1	74.79	2024-07-13
2	1	-224.1	2024-05-13
3	1	-128.9	2024-01-25
*/
Invoking: `sql_db_query` with `{'query': 'SELECT SUM(amount) AS total_amount FROM transactions WHERE account_id = 1'}`


[(-965.7,)]
Invoking: `sql_db_query` with `{'query': 'SELECT SUM(amount) AS total_amount FROM transactions WHERE account_id = 2'}`


[(743.13,)]테디의 거래 합계는 -965.7이고, 셜리의 거래 합계는 743.13입니다.

> Finished chain.


{'input': '테디와 셜리의 transaction 의 합계를 구하고 비교하세요',
 'output': '테디의 거래 합계는 -965.7이고, 셜리의 거래 합계는 743.13입니다.'}